In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
gcs = GraphOfConvexSets()

v1 = gcs.add_vertex("v1")
x1 = v1.add_variable(2)
c1 = np.array([0, .5])
v1.add_constraint(cp.norm(x1 - c1, 2) <= .5)

v2 = gcs.add_vertex("v2")
x2 = v2.add_variable(2)
c2 = np.array([0, -1])
v2.add_constraint(cp.norm(x2 - c2, 2) <= .5)

v3 = gcs.add_vertex("v3")
x3 = v3.add_variable(2)
c3 = np.array([0, -2.5])
v3.add_constraint(cp.norm(x3 - c3, 2) <= .5)

v4 = gcs.add_vertex("v4")
x4 = v4.add_variable(2)
c4 = np.array([3, 2.5])
v4.add_constraint(cp.norm(x4 - c4, np.inf) <= .5)

v5 = gcs.add_vertex("v5")
x5 = v5.add_variable(2)
c5 = np.array([3, -1])
D5 = np.diag([2, .5])
v5.add_constraint(cp.norm(D5 @ (x5 - c5), np.inf) <= 1)

facilities = [3, 4]
customers = [0, 1, 2]

In [ ]:
for facility in facilities:
    for customer in customers:
        tail = gcs.vertices[facility]
        head = gcs.vertices[customer]
        edge = gcs.add_edge(tail, head)
        edge.add_cost(cp.norm(tail.variables[0] - head.variables[0], 2))

In [ ]:
gcs.graphviz()

In [ ]:
prob = gcs.solve_facility_location(facilities)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)
print('Vertex binaries:', {vertex.name: np.round(vertex.y.value, 4) for vertex in gcs.vertices})
print('Edge binaries:', {edge.name: np.round(edge.y.value, 4) for edge in gcs.edges})

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()
gcs.plot_subgraph_2d()
# plt.savefig('facility_location.pdf')

# From ILP

In [ ]:
ilp_constraints = []
yv = gcs.vertex_binaries()
ye = gcs.edge_binaries()

for i, v in enumerate(gcs.vertices):
    if i in facilities:
        ilp_constraints.append(yv[i] <= 1)
    elif i in customers:
        ilp_constraints.append(yv[i] == 1)
        inc_edges = gcs.incoming_indices(v)
        ilp_constraints.append(sum(ye[inc_edges]) == 1)
        
for k, edge in enumerate(gcs.edges):
    i = gcs.vertex_index(edge.tail)
    ilp_constraints.append(yv[i] >= ye[k])

In [ ]:
prob = gcs.solve_from_ilp(ilp_constraints)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)